In [1]:
import rpy2

In [2]:
print(rpy2.__version__)

2.9.4


In [19]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pandas

  Using cached pytz-2021.3-py2.py3-none-any.whl (503 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install torch

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import csv

# Final Project

## Getting data

In [4]:
train_df = pd.read_csv('train_preprocessed.csv')
test_df = pd.read_csv('test_preprocessed.csv')
calendar_df = pd.read_csv('calendar_preprocessed.csv')
prices_df = pd.read_csv('sell_prices_afcs2021.csv')

## Preprocess some more

In [5]:
train_df.columns

Index(['Unnamed: 0', 'Day', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
       ...
       'month', 'year', 'd', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'Event', 'Day_type'],
      dtype='object', length=839)

In [6]:
product_list = prices_df['item_id'].to_list()

product_list_unique = []
[product_list_unique.append(x) for x in product_list if x not in product_list_unique] 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [7]:
def rename_products(dataset):
    
    index = 0
    for column in dataset.columns:
        if column[0] == 'V':

            dataset.rename(columns={column: product_list_unique[index]}, inplace = True)
            index += 1

In [8]:
def clean_data(dataset):
    
    dataset = dataset.drop("Unnamed: 0", axis=1)
    rename_products(dataset)
    dataset = dataset.drop(["Day", "Date", "weekday", "year", "d", "event_name_1", "event_type_1",
                           "event_name_2", "event_type_2", "snap_CA", "Day_type"], axis=1)
    
    return dataset

train_df = clean_data(train_df)
test_df = clean_data(test_df)

In [9]:
test_df

,FOODS_3_001,FOODS_3_002,FOODS_3_003,FOODS_3_004,FOODS_3_005,FOODS_3_006,FOODS_3_007,FOODS_3_008,FOODS_3_009,FOODS_3_010,...,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827,wm_yr_wk,wday,month,Event
0,1,0,0,0,1,0,8,0,1,2,...,0,0,0,2,1,10,11617,3,5,0
1,0,0,0,0,1,1,0,0,0,0,...,2,0,0,0,0,0,11617,4,5,0
2,0,0,1,0,2,1,3,0,0,0,...,1,1,0,0,1,2,11617,5,5,0
3,1,0,0,0,0,0,0,0,1,1,...,4,0,0,2,5,0,11617,6,5,0
4,0,0,0,1,2,1,15,0,2,0,...,3,0,0,2,1,0,11617,7,5,0
5,1,0,0,0,0,1,0,0,0,0,...,2,0,0,0,0,3,11618,1,5,0
6,0,0,0,1,2,3,12,0,2,2,...,1,0,0,3,1,0,11618,2,5,0
7,0,0,0,0,1,1,6,0,0,0,...,2,0,2,1,1,1,11618,3,5,3
8,0,0,1,0,4,0,14,0,0,0,...,2,1,0,1,3,2,11618,4,5,0
9,0,2,1,0,0,2,17,0,0,0,...,3,0,0,1,1,1,11618,5,6,0


In [10]:
for column in test_df.columns:
    print(column)

FOODS_3_001
FOODS_3_002
FOODS_3_003
FOODS_3_004
FOODS_3_005
FOODS_3_006
FOODS_3_007
FOODS_3_008
FOODS_3_009
FOODS_3_010
FOODS_3_011
FOODS_3_012
FOODS_3_013
FOODS_3_014
FOODS_3_015
FOODS_3_016
FOODS_3_017
FOODS_3_018
FOODS_3_019
FOODS_3_020
FOODS_3_021
FOODS_3_022
FOODS_3_023
FOODS_3_024
FOODS_3_025
FOODS_3_026
FOODS_3_027
FOODS_3_028
FOODS_3_029
FOODS_3_030
FOODS_3_031
FOODS_3_032
FOODS_3_033
FOODS_3_034
FOODS_3_035
FOODS_3_036
FOODS_3_037
FOODS_3_038
FOODS_3_039
FOODS_3_040
FOODS_3_041
FOODS_3_042
FOODS_3_043
FOODS_3_044
FOODS_3_045
FOODS_3_046
FOODS_3_047
FOODS_3_048
FOODS_3_049
FOODS_3_050
FOODS_3_051
FOODS_3_053
FOODS_3_054
FOODS_3_055
FOODS_3_056
FOODS_3_057
FOODS_3_058
FOODS_3_059
FOODS_3_060
FOODS_3_061
FOODS_3_062
FOODS_3_063
FOODS_3_064
FOODS_3_065
FOODS_3_066
FOODS_3_067
FOODS_3_068
FOODS_3_069
FOODS_3_070
FOODS_3_071
FOODS_3_072
FOODS_3_073
FOODS_3_074
FOODS_3_075
FOODS_3_076
FOODS_3_077
FOODS_3_078
FOODS_3_079
FOODS_3_080
FOODS_3_081
FOODS_3_082
FOODS_3_083
FOODS_3_084
FOOD

In [11]:
test_df

,FOODS_3_001,FOODS_3_002,FOODS_3_003,FOODS_3_004,FOODS_3_005,FOODS_3_006,FOODS_3_007,FOODS_3_008,FOODS_3_009,FOODS_3_010,...,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827,wm_yr_wk,wday,month,Event
0,1,0,0,0,1,0,8,0,1,2,...,0,0,0,2,1,10,11617,3,5,0
1,0,0,0,0,1,1,0,0,0,0,...,2,0,0,0,0,0,11617,4,5,0
2,0,0,1,0,2,1,3,0,0,0,...,1,1,0,0,1,2,11617,5,5,0
3,1,0,0,0,0,0,0,0,1,1,...,4,0,0,2,5,0,11617,6,5,0
4,0,0,0,1,2,1,15,0,2,0,...,3,0,0,2,1,0,11617,7,5,0
5,1,0,0,0,0,1,0,0,0,0,...,2,0,0,0,0,3,11618,1,5,0
6,0,0,0,1,2,3,12,0,2,2,...,1,0,0,3,1,0,11618,2,5,0
7,0,0,0,0,1,1,6,0,0,0,...,2,0,2,1,1,1,11618,3,5,3
8,0,0,1,0,4,0,14,0,0,0,...,2,1,0,1,3,2,11618,4,5,0
9,0,2,1,0,0,2,17,0,0,0,...,3,0,0,1,1,1,11618,5,6,0


In [12]:
################################ price df

In [13]:
prices_df

,store_id,item_id,wm_yr_wk,sell_price
0,TX_3,FOODS_3_001,11101,2.28
1,TX_3,FOODS_3_001,11102,2.28
2,TX_3,FOODS_3_001,11103,2.28
3,TX_3,FOODS_3_001,11104,2.28
4,TX_3,FOODS_3_001,11105,2.28
...,...,...,...,...
185255,TX_3,FOODS_3_827,11617,1.00
185256,TX_3,FOODS_3_827,11618,1.00
185257,TX_3,FOODS_3_827,11619,1.00
185258,TX_3,FOODS_3_827,11620,1.00


In [14]:
prices_df = prices_df.pivot(index = 'wm_yr_wk', columns = 'item_id', values = 'sell_price')

In [15]:
prices_df

item_id,FOODS_3_001,FOODS_3_002,FOODS_3_003,FOODS_3_004,FOODS_3_005,FOODS_3_006,FOODS_3_007,FOODS_3_008,FOODS_3_009,FOODS_3_010,...,FOODS_3_818,FOODS_3_819,FOODS_3_820,FOODS_3_821,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827
wm_yr_wk,,,,,,,,,,,,,,,,,,,,,
11101,2.28,NaN,NaN,NaN,1.68,NaN,0.20,NaN,NaN,NaN,...,NaN,1.98,1.58,NaN,3.98,NaN,2.47,4.00,NaN,NaN
11102,2.28,NaN,NaN,NaN,1.68,NaN,0.20,NaN,NaN,NaN,...,NaN,1.98,1.58,NaN,3.98,NaN,2.47,4.00,NaN,NaN
11103,2.28,NaN,NaN,NaN,1.98,NaN,0.16,NaN,NaN,NaN,...,NaN,1.76,1.58,NaN,3.98,NaN,2.47,4.00,NaN,NaN
11104,2.28,NaN,NaN,NaN,1.98,NaN,0.07,NaN,NaN,NaN,...,NaN,1.76,1.58,NaN,3.98,2.50,2.47,4.00,NaN,NaN
11105,2.28,NaN,NaN,NaN,1.98,NaN,0.16,NaN,NaN,NaN,...,NaN,1.76,1.58,NaN,3.98,2.50,2.47,4.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11617,2.50,2.56,3.64,5.98,2.28,7.88,0.25,0.74,2.28,2.84,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
11618,2.50,2.56,3.64,5.98,2.28,7.88,0.25,0.74,2.28,2.84,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0
11619,2.50,2.56,3.64,5.98,2.28,7.88,0.25,0.74,2.28,2.84,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0


In [16]:
prices_train_df = prices_df.loc[11101:11613]
prices_test_df = prices_df.loc[11616:]

In [17]:
prices_train_df
prices_test_df

item_id,FOODS_3_001,FOODS_3_002,FOODS_3_003,FOODS_3_004,FOODS_3_005,FOODS_3_006,FOODS_3_007,FOODS_3_008,FOODS_3_009,FOODS_3_010,...,FOODS_3_818,FOODS_3_819,FOODS_3_820,FOODS_3_821,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827
wm_yr_wk,,,,,,,,,,,,,,,,,,,,,
11616,2.5,2.56,3.64,5.98,2.28,7.88,0.25,0.74,2.28,2.84,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
11617,2.5,2.56,3.64,5.98,2.28,7.88,0.25,0.74,2.28,2.84,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
11618,2.5,2.56,3.64,5.98,2.28,7.88,0.25,0.74,2.28,2.84,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0
11619,2.5,2.56,3.64,5.98,2.28,7.88,0.25,0.74,2.28,2.84,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
11620,2.5,2.56,3.64,5.98,2.28,7.88,0.25,0.74,2.28,2.84,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
11621,2.5,2.56,3.64,5.98,2.28,7.88,0.25,0.74,2.28,2.84,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0


In [18]:
full_train_dataset = pd.merge(train_df, prices_train_df, on=['wm_yr_wk'])
full_test_dataset = pd.merge(test_df, prices_test_df, on=['wm_yr_wk'])

In [19]:
full_train_dataset

,FOODS_3_001_x,FOODS_3_002_x,FOODS_3_003_x,FOODS_3_004_x,FOODS_3_005_x,FOODS_3_006_x,FOODS_3_007_x,FOODS_3_008_x,FOODS_3_009_x,FOODS_3_010_x,...,FOODS_3_818_y,FOODS_3_819_y,FOODS_3_820_y,FOODS_3_821_y,FOODS_3_822_y,FOODS_3_823_y,FOODS_3_824_y,FOODS_3_825_y,FOODS_3_826_y,FOODS_3_827_y
0,0,0,0,0,0,0,27,0,0,0,...,NaN,1.98,1.58,NaN,3.98,NaN,2.47,4.00,NaN,NaN
1,2,0,0,0,1,0,27,0,0,0,...,NaN,1.98,1.58,NaN,3.98,NaN,2.47,4.00,NaN,NaN
2,1,0,0,0,1,0,8,0,0,0,...,NaN,1.98,1.58,NaN,3.98,NaN,2.47,4.00,NaN,NaN
3,3,0,0,0,0,0,0,0,0,0,...,NaN,1.98,1.58,NaN,3.98,NaN,2.47,4.00,NaN,NaN
4,0,0,0,0,0,0,0,0,0,0,...,NaN,1.98,1.58,NaN,3.98,NaN,2.47,4.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,0,0,0,0,0,0,0,0,1,0,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
1909,0,0,0,0,1,1,0,0,0,0,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
1910,5,0,1,0,0,1,0,0,0,0,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
1911,2,0,0,0,4,1,0,0,0,0,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0


In [20]:
full_test_dataset

,FOODS_3_001_x,FOODS_3_002_x,FOODS_3_003_x,FOODS_3_004_x,FOODS_3_005_x,FOODS_3_006_x,FOODS_3_007_x,FOODS_3_008_x,FOODS_3_009_x,FOODS_3_010_x,...,FOODS_3_818_y,FOODS_3_819_y,FOODS_3_820_y,FOODS_3_821_y,FOODS_3_822_y,FOODS_3_823_y,FOODS_3_824_y,FOODS_3_825_y,FOODS_3_826_y,FOODS_3_827_y
0,1,0,0,0,1,0,8,0,1,2,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
1,0,0,0,0,1,1,0,0,0,0,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
2,0,0,1,0,2,1,3,0,0,0,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
3,1,0,0,0,0,0,0,0,1,1,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
4,0,0,0,1,2,1,15,0,2,0,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
5,1,0,0,0,0,1,0,0,0,0,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0
6,0,0,0,1,2,3,12,0,2,2,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0
7,0,0,0,0,1,1,6,0,0,0,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0
8,0,0,1,0,4,0,14,0,0,0,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0
9,0,2,1,0,0,2,17,0,0,0,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0


In [21]:
for column in full_train_dataset.columns:
    print(column)

FOODS_3_001_x
FOODS_3_002_x
FOODS_3_003_x
FOODS_3_004_x
FOODS_3_005_x
FOODS_3_006_x
FOODS_3_007_x
FOODS_3_008_x
FOODS_3_009_x
FOODS_3_010_x
FOODS_3_011_x
FOODS_3_012_x
FOODS_3_013_x
FOODS_3_014_x
FOODS_3_015_x
FOODS_3_016_x
FOODS_3_017_x
FOODS_3_018_x
FOODS_3_019_x
FOODS_3_020_x
FOODS_3_021_x
FOODS_3_022_x
FOODS_3_023_x
FOODS_3_024_x
FOODS_3_025_x
FOODS_3_026_x
FOODS_3_027_x
FOODS_3_028_x
FOODS_3_029_x
FOODS_3_030_x
FOODS_3_031_x
FOODS_3_032_x
FOODS_3_033_x
FOODS_3_034_x
FOODS_3_035_x
FOODS_3_036_x
FOODS_3_037_x
FOODS_3_038_x
FOODS_3_039_x
FOODS_3_040_x
FOODS_3_041_x
FOODS_3_042_x
FOODS_3_043_x
FOODS_3_044_x
FOODS_3_045_x
FOODS_3_046_x
FOODS_3_047_x
FOODS_3_048_x
FOODS_3_049_x
FOODS_3_050_x
FOODS_3_051_x
FOODS_3_053_x
FOODS_3_054_x
FOODS_3_055_x
FOODS_3_056_x
FOODS_3_057_x
FOODS_3_058_x
FOODS_3_059_x
FOODS_3_060_x
FOODS_3_061_x
FOODS_3_062_x
FOODS_3_063_x
FOODS_3_064_x
FOODS_3_065_x
FOODS_3_066_x
FOODS_3_067_x
FOODS_3_068_x
FOODS_3_069_x
FOODS_3_070_x
FOODS_3_071_x
FOODS_3_072_x
FOODS_

In [22]:
full_train_dataset = full_train_dataset.drop(["wm_yr_wk"], axis=1)
full_test_dataset = full_test_dataset.drop(["wm_yr_wk"], axis=1)

In [23]:
for column in full_train_dataset.columns:
    print(column)

FOODS_3_001_x
FOODS_3_002_x
FOODS_3_003_x
FOODS_3_004_x
FOODS_3_005_x
FOODS_3_006_x
FOODS_3_007_x
FOODS_3_008_x
FOODS_3_009_x
FOODS_3_010_x
FOODS_3_011_x
FOODS_3_012_x
FOODS_3_013_x
FOODS_3_014_x
FOODS_3_015_x
FOODS_3_016_x
FOODS_3_017_x
FOODS_3_018_x
FOODS_3_019_x
FOODS_3_020_x
FOODS_3_021_x
FOODS_3_022_x
FOODS_3_023_x
FOODS_3_024_x
FOODS_3_025_x
FOODS_3_026_x
FOODS_3_027_x
FOODS_3_028_x
FOODS_3_029_x
FOODS_3_030_x
FOODS_3_031_x
FOODS_3_032_x
FOODS_3_033_x
FOODS_3_034_x
FOODS_3_035_x
FOODS_3_036_x
FOODS_3_037_x
FOODS_3_038_x
FOODS_3_039_x
FOODS_3_040_x
FOODS_3_041_x
FOODS_3_042_x
FOODS_3_043_x
FOODS_3_044_x
FOODS_3_045_x
FOODS_3_046_x
FOODS_3_047_x
FOODS_3_048_x
FOODS_3_049_x
FOODS_3_050_x
FOODS_3_051_x
FOODS_3_053_x
FOODS_3_054_x
FOODS_3_055_x
FOODS_3_056_x
FOODS_3_057_x
FOODS_3_058_x
FOODS_3_059_x
FOODS_3_060_x
FOODS_3_061_x
FOODS_3_062_x
FOODS_3_063_x
FOODS_3_064_x
FOODS_3_065_x
FOODS_3_066_x
FOODS_3_067_x
FOODS_3_068_x
FOODS_3_069_x
FOODS_3_070_x
FOODS_3_071_x
FOODS_3_072_x
FOODS_

In [24]:
print(full_train_dataset['wday'].value_counts())
print(full_train_dataset['month'].value_counts())
print(full_train_dataset['Event'].value_counts())

1    274
2    274
3    273
4    273
5    273
6    273
7    273
Name: wday, dtype: int64
3     186
4     174
2     170
1     158
5     155
7     155
8     155
10    155
12    155
6     150
9     150
11    150
Name: month, dtype: int64
0    1759
4      52
3      51
2      35
1      16
Name: Event, dtype: int64


## Create dummy variables

In [25]:
full_train_dataset = pd.get_dummies(full_train_dataset, prefix=['wday_', 'month_', 'Event_'], columns=['wday', 'month', 'Event'])

In [26]:
full_train_dataset

,FOODS_3_001_x,FOODS_3_002_x,FOODS_3_003_x,FOODS_3_004_x,FOODS_3_005_x,FOODS_3_006_x,FOODS_3_007_x,FOODS_3_008_x,FOODS_3_009_x,FOODS_3_010_x,...,month__8,month__9,month__10,month__11,month__12,Event__0,Event__1,Event__2,Event__3,Event__4
0,0,0,0,0,0,0,27,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2,0,0,0,1,0,27,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,0,0,0,1,0,8,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1909,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1910,5,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1911,2,0,0,0,4,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [27]:
for column in full_train_dataset.columns:
    print(column)
    
len(full_train_dataset.columns)

FOODS_3_001_x
FOODS_3_002_x
FOODS_3_003_x
FOODS_3_004_x
FOODS_3_005_x
FOODS_3_006_x
FOODS_3_007_x
FOODS_3_008_x
FOODS_3_009_x
FOODS_3_010_x
FOODS_3_011_x
FOODS_3_012_x
FOODS_3_013_x
FOODS_3_014_x
FOODS_3_015_x
FOODS_3_016_x
FOODS_3_017_x
FOODS_3_018_x
FOODS_3_019_x
FOODS_3_020_x
FOODS_3_021_x
FOODS_3_022_x
FOODS_3_023_x
FOODS_3_024_x
FOODS_3_025_x
FOODS_3_026_x
FOODS_3_027_x
FOODS_3_028_x
FOODS_3_029_x
FOODS_3_030_x
FOODS_3_031_x
FOODS_3_032_x
FOODS_3_033_x
FOODS_3_034_x
FOODS_3_035_x
FOODS_3_036_x
FOODS_3_037_x
FOODS_3_038_x
FOODS_3_039_x
FOODS_3_040_x
FOODS_3_041_x
FOODS_3_042_x
FOODS_3_043_x
FOODS_3_044_x
FOODS_3_045_x
FOODS_3_046_x
FOODS_3_047_x
FOODS_3_048_x
FOODS_3_049_x
FOODS_3_050_x
FOODS_3_051_x
FOODS_3_053_x
FOODS_3_054_x
FOODS_3_055_x
FOODS_3_056_x
FOODS_3_057_x
FOODS_3_058_x
FOODS_3_059_x
FOODS_3_060_x
FOODS_3_061_x
FOODS_3_062_x
FOODS_3_063_x
FOODS_3_064_x
FOODS_3_065_x
FOODS_3_066_x
FOODS_3_067_x
FOODS_3_068_x
FOODS_3_069_x
FOODS_3_070_x
FOODS_3_071_x
FOODS_3_072_x
FOODS_

1670

### Models

In [28]:
full_train_dataset
full_test_dataset

,FOODS_3_001_x,FOODS_3_002_x,FOODS_3_003_x,FOODS_3_004_x,FOODS_3_005_x,FOODS_3_006_x,FOODS_3_007_x,FOODS_3_008_x,FOODS_3_009_x,FOODS_3_010_x,...,FOODS_3_818_y,FOODS_3_819_y,FOODS_3_820_y,FOODS_3_821_y,FOODS_3_822_y,FOODS_3_823_y,FOODS_3_824_y,FOODS_3_825_y,FOODS_3_826_y,FOODS_3_827_y
0,1,0,0,0,1,0,8,0,1,2,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
1,0,0,0,0,1,1,0,0,0,0,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
2,0,0,1,0,2,1,3,0,0,0,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
3,1,0,0,0,0,0,0,0,1,1,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
4,0,0,0,1,2,1,15,0,2,0,...,2.98,1.98,1.98,4.98,3.98,2.98,2.48,3.98,1.28,1.0
5,1,0,0,0,0,1,0,0,0,0,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0
6,0,0,0,1,2,3,12,0,2,2,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0
7,0,0,0,0,1,1,6,0,0,0,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0
8,0,0,1,0,4,0,14,0,0,0,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0
9,0,2,1,0,0,2,17,0,0,0,...,2.98,1.98,1.98,4.98,2.50,2.98,2.48,3.98,1.28,1.0


In [29]:
# Try LSTM
# Calculate metrics
# Normalize/Standardize data
# Try other models

In [30]:
class LSTM(nn.Module):
    
    def __init__(self, n_inputs, n_hidden, n_classes, batch_size):
        
        super().__init__()
        
        self.seq_length = n_inputs
        self.hidden_dim = n_hidden
        self.output_dim = n_classes
        self.batch_size = batch_size
        
        self.W_gx = nn.Parameter(nn.init.kaiming_normal_(torch.randn(self.hidden_dim, self.seq_length)))
        self.W_gh = nn.Parameter(nn.init.kaiming_normal_(torch.randn(self.hidden_dim, self.hidden_dim)))
        self.b_g = nn.Parameter(torch.zeros(self.hidden_dim, 1))
        
        self.W_ix = nn.Parameter(nn.init.kaiming_normal_(torch.randn(self.hidden_dim, self.seq_length)))
        self.W_ih = nn.Parameter(nn.init.kaiming_normal_(torch.randn(self.hidden_dim, self.hidden_dim)))
        self.b_i = nn.Parameter(torch.zeros(self.hidden_dim, 1))
        
        self.W_fx = nn.Parameter(nn.init.kaiming_normal_(torch.randn(self.hidden_dim, self.seq_length)))
        self.W_fh = nn.Parameter(nn.init.kaiming_normal_(torch.randn(self.hidden_dim, self.hidden_dim)))
        self.b_f = nn.Parameter(torch.zeros(self.hidden_dim, 1))
        
        self.W_ox = nn.Parameter(nn.init.kaiming_normal_(torch.randn(self.hidden_dim, self.seq_length)))
        self.W_oh = nn.Parameter(nn.init.kaiming_normal_(torch.randn(self.hidden_dim, self.hidden_dim)))
        self.b_o = nn.Parameter(torch.zeros(self.hidden_dim, 1))
        
        self.W_ph = nn.Parameter(nn.init.kaiming_normal_(torch.randn(self.output_dim, self.hidden_dim)))
        self.b_p = nn.Parameter(torch.zeros(self.output_dim, 1))
        
        self.tanh = nn.Tanh()
        self.sigmoid= nn.Sigmoid()
        
    def lstm_cell(self, x_t, h_t_old, c_t_old):
        
        g_t = self.tanh(torch.matmul(self.W_gx, x_t) + torch.matmul(self.W_gh, h_t_old) + self.b_g)
        i_t = self.sigmoid(torch.matmul(self.W_ix, x_t) + torch.matmul(self.W_ih, h_t_old) + self.b_i)
        f_t = self.sigmoid(torch.matmul(self.W_fx, x_t) + torch.matmul(self.W_fh, h_t_old) + self.b_f)
        o_t = self.sigmoid(torch.matmul(self.W_ox, x_t) + torch.matmul(self.W_oh, h_t_old) + self.b_o)
        
        c_t = torch.mul(i_t, g_t) + torch.mul(f_t, c_t_old)
        h_t = torch.mul(o_t, self.tanh(c_t))
        
        return h_t, c_t
    
    def forward(self, x):
        
        predictions = torch.zeros(x.shape[0], self.output_dim)
#         print(predictions.shape)
        h_t, c_t = nn.Parameter(torch.zeros(self.hidden_dim, 1)), nn.Parameter(torch.zeros(self.hidden_dim, 1))
        
        for i in range(x.shape[0]):
            
            x_t = x[i]
            x_t = x_t.reshape(x_t.shape[0], 1)
#             print(x_t.shape)
#             x_t = x_t.reshape(x_t.shape[0], x_t.shape[1], 1)
            h_t, c_t = self.lstm_cell(x_t, h_t, c_t)
            
            p_t = (torch.matmul(self.W_ph, h_t) + self.b_p)
            p_t = torch.transpose(p_t, 0, 1)
#             print("p_t: ", p_t.shape)

            predictions[i] = p_t
        
#         print(predictions.shape)
#         predictions = predictions.reshape(-1, predictions.shape[1], predictions.shape[2])
        return predictions

In [31]:
def train(dataset):
    
    np.random.seed(42)
    torch.manual_seed(42)
    
    n_inputs, n_outputs, n_hidden = 1670, 823, 2048
#     n_inputs, n_outputs, n_hidden = 823, 823, 1024
    batch_size = 16
    
    dataset_train = dataset[:1344]
    dataset_validation = dataset[1344:]
    
    loss_module = nn.MSELoss(reduction = 'sum')
    model = LSTM(n_inputs, n_hidden, n_outputs, batch_size)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    all_mape = []
    all_X_validation_rolled, all_predicted_labels_validation, validation_residuals = [], [], []
    
#     target_train = np.zeros((dataset_train.shape[0], n_features))
    
    for epoch in range(3):
        
        model.train()
        num_batches = int(dataset_train.shape[0]/batch_size)
        
        for batch in range(num_batches):
            
            X_train = torch.tensor(dataset_train[batch*batch_size:(batch+1)*batch_size]).float()
            
            model.zero_grad()
            
#             print(X_train.shape)
            predicted_labels = model(X_train)
            
            X_train_rolled = torch.roll(X_train, -1, 0)
#             print("Y: ", X_train_rolled[:15, :823])
#             print("Predictions: ", predicted_labels[:15, :])
            
            average_loss = torch.sqrt(loss_module(predicted_labels[:15, :], X_train_rolled[:15, :823]))
            print(average_loss, '\n')
            
            average_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
            optimizer.step()
            
        model.eval()
        num_batches = int(dataset_validation.shape[0]/batch_size)
        
        with torch.no_grad():
            print("VALIDATION: ")
            
            for batch in range(num_batches):

                X_validation = torch.tensor(dataset_validation[batch*batch_size:(batch+1)*batch_size]).float()

                predicted_labels = model(X_validation)

                X_validation_rolled = torch.roll(X_validation, -1, 0)
#                 print("Y: ", X_validation_rolled[:15, :823])
#                 print("Predictions: ", predicted_labels[:15, :])

                average_loss = torch.sqrt(loss_module(predicted_labels[:15, :], X_validation_rolled[:15, :823]))
                print(average_loss, '\n')

In [32]:
# To solve: NaN in empty values!!!!!
# Then also add external features

full_train_dataset = (full_train_dataset-full_train_dataset.min())/(full_train_dataset.max()-full_train_dataset.min())
full_train_dataset

nan_values = full_train_dataset.isna()
nan_columns = nan_values.any()

columns_with_nan = full_train_dataset.columns[nan_columns].tolist()
columns_with_nan

for column in columns_with_nan:
    full_train_dataset[column] = full_train_dataset[column].interpolate().fillna(method='bfill')
    if full_train_dataset[column].isnull().values.any():
        full_train_dataset[column] = 0.5

In [33]:
full_train_tensor = torch.tensor(full_train_dataset.values)
# full_train_tensor = full_train_tensor[:, :823]

train(full_train_tensor)

C:\Users\roxan\Anaconda3\envs\my-r-env\lib\site-packages\ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor(39.4455, grad_fn=<SqrtBackward0>) 

tensor(29.8198, grad_fn=<SqrtBackward0>) 

tensor(23.6811, grad_fn=<SqrtBackward0>) 

tensor(19.6580, grad_fn=<SqrtBackward0>) 

tensor(18.1277, grad_fn=<SqrtBackward0>) 

tensor(16.3628, grad_fn=<SqrtBackward0>) 

tensor(16.5880, grad_fn=<SqrtBackward0>) 

tensor(16.1325, grad_fn=<SqrtBackward0>) 

tensor(16.4347, grad_fn=<SqrtBackward0>) 

tensor(15.3152, grad_fn=<SqrtBackward0>) 

tensor(14.9027, grad_fn=<SqrtBackward0>) 

tensor(14.8562, grad_fn=<SqrtBackward0>) 

tensor(15.9832, grad_fn=<SqrtBackward0>) 

tensor(14.6799, grad_fn=<SqrtBackward0>) 

tensor(15.6988, grad_fn=<SqrtBackward0>) 

tensor(15.7494, grad_fn=<SqrtBackward0>) 

tensor(15.7055, grad_fn=<SqrtBackward0>) 

tensor(15.0278, grad_fn=<SqrtBackward0>) 

tensor(14.4262, grad_fn=<SqrtBackward0>) 

tensor(13.7229, grad_fn=<SqrtBackward0>) 

tensor(13.7098, grad_fn=<SqrtBackward0>) 

tensor(13.4046, grad_fn=<SqrtBackward0>) 

tensor(12.0910, grad_fn=<SqrtBackward0>) 

tensor(13.6

C:\Users\roxan\Anaconda3\envs\my-r-env\lib\site-packages\ipykernel_launcher.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor(13.4206) 

tensor(13.9175) 

tensor(13.5422) 

tensor(13.7056) 

tensor(14.4963) 

tensor(14.8932) 

tensor(14.9919) 

tensor(15.1763) 

tensor(15.5257) 

tensor(14.7864) 

tensor(16.6537) 

tensor(15.6637) 

tensor(16.3253) 

tensor(14.7826) 

tensor(17.6159) 

tensor(14.4934) 

tensor(15.8762) 

tensor(15.5464) 

tensor(16.6387) 

tensor(15.8846) 

tensor(16.5695) 

tensor(15.6096) 

tensor(16.0977) 

tensor(15.5264) 

tensor(15.3575) 

tensor(15.2832) 

tensor(15.7609) 

tensor(15.2445) 

tensor(15.2659) 

tensor(16.2904) 

tensor(16.1743) 

tensor(16.4911) 

tensor(16.2678) 

tensor(17.0738) 

tensor(16.2881, grad_fn=<SqrtBackward0>) 

tensor(15.5813, grad_fn=<SqrtBackward0>) 

tensor(15.3184, grad_fn=<SqrtBackward0>) 

tensor(14.1238, grad_fn=<SqrtBackward0>) 

tensor(13.8158, grad_fn=<SqrtBackward0>) 

tensor(12.1731, grad_fn=<SqrtBackward0>) 

tensor(12.4453, grad_fn=<SqrtBackward0>) 

tensor(12.0706, grad_fn=<SqrtBackward0>) 

tensor(12.5180, grad_fn=<SqrtBackward0>) 

t

tensor(13.6732, grad_fn=<SqrtBackward0>) 

tensor(12.1968, grad_fn=<SqrtBackward0>) 

tensor(13.0747, grad_fn=<SqrtBackward0>) 

tensor(13.2550, grad_fn=<SqrtBackward0>) 

tensor(13.1039, grad_fn=<SqrtBackward0>) 

tensor(12.3143, grad_fn=<SqrtBackward0>) 

VALIDATION: 
tensor(14.2477) 

tensor(12.7895) 

tensor(13.1863) 

tensor(12.9527) 

tensor(13.0495) 

tensor(13.7402) 

tensor(14.2414) 

tensor(14.2715) 

tensor(14.4523) 

tensor(14.6603) 

tensor(13.9380) 

tensor(15.9147) 

tensor(14.8459) 

tensor(15.5103) 

tensor(13.9820) 

tensor(16.8644) 

tensor(13.5847) 

tensor(15.0746) 

tensor(14.7980) 

tensor(15.8043) 

tensor(14.9515) 

tensor(15.7019) 

tensor(14.7520) 

tensor(15.2701) 

tensor(14.5724) 

tensor(14.2936) 

tensor(14.2221) 

tensor(14.6234) 

tensor(14.0196) 

tensor(13.9563) 

tensor(14.9966) 

tensor(14.9088) 

tensor(15.2435) 

tensor(14.9034) 

tensor(15.6718) 

